In [1]:
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
gpu_available = tf.test.is_gpu_available() 
print(gpu_available)

2023-08-08 11:08:28.872243: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 11:08:29.710152: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


2023-08-08 11:08:31.156587: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-08 11:08:31.156620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: langchain
2023-08-08 11:08:31.156627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: langchain
2023-08-08 11:08:31.156808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 520.56.6
2023-08-08 11:08:31.156826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 520.56.6
2023-08-08 11:08:31.156830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 520.56.6


In [1]:
import pandas as pd
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic



def create_landmark_frame_df(results,frame,xyz):
    """
    xyz takes the results from mediapipe and creates a dataframe of the landmark
    
    inputs:
        results: mediapipe results object
        frame: frame number
        xyz: dataframe wof the xyz example data
    
    
    """
    
    #we want the values and rows for every type landmark index so we need skeleton
    xyz_skel = xyz[['type','landmark_index']].drop_duplicates() \
    .reset_index(drop=True).copy()
    
    pose = pd.DataFrame()
    face = pd.DataFrame()
    left_hand = pd.DataFrame()
    right_hand = pd.DataFrame()
    
    if results.face_landmarks is not None:
        for i, point in enumerate(results.face_landmarks.landmark):
            face.loc[i, ['x','y','z']] = [point.x, point.y, point.z]

    if results.pose_landmarks is not None:
        for i, point in enumerate(results.pose_landmarks.landmark):
            pose.loc[i, ['x','y','z']] = [point.x, point.y, point.z]


    if results.left_hand_landmarks is not None:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            left_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]


    if results.right_hand_landmarks is not None:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            right_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]

    face = face.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='face')

    pose = pose.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='pose')

    left_hand = left_hand.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='left_hand')

    right_hand = right_hand.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='right_hand')

    landmark = pd.concat([face,pose,left_hand,right_hand]).reset_index(drop=True)
    #merge with lanndmark
    landmark = xyz_skel.merge(landmark, on=['type','landmark_index'], how='left')
    #assign frames to make it unique
    landmark = landmark.assign(frame = frame)
    return landmark


2023-08-08 08:32:38.296241: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 08:32:39.098777: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# working code

In [2]:
def do_capture_web(xyz):
    all_landmarks = []

    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
        frame = 0  
        while cap.isOpened():
                #take frame and increment it
            frame+=1
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                    # If loading a video, use 'break' instead of 'continue'.
                continue

                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            #create landmark dataframe
            landmark_df = create_landmark_frame_df(results, frame, xyz)
            all_landmarks.append(landmark_df)

            # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())

            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_pose_landmarks_style())

            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())

            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())
            
            # Iterate through the tracked landmarks
        



                # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break
                

    return all_landmarks    

if __name__ == "__main__":
    xyz = pd.read_parquet('./2044/635217.parquet')
    landmark = do_capture_web(xyz)
    landmark = pd.concat(landmark).reset_index(drop=True)
    landmark.to_parquet('output2.parquet')



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


ValueError: cannot convert float NaN to integer

# dummy code

In [3]:
import asyncio
import pandas as pd
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic



def create_landmark_frame_df(results,frame,xyz):
    """
    xyz takes the results from mediapipe and creates a dataframe of the landmark
    
    inputs:
        results: mediapipe results object
        frame: frame number
        xyz: dataframe wof the xyz example data
    
    
    """
    
    #we want the values and rows for every type landmark index so we need skeleton
    xyz_skel = xyz[['type','landmark_index']].drop_duplicates() \
    .reset_index(drop=True).copy()
    
    pose = pd.DataFrame()
    face = pd.DataFrame()
    left_hand = pd.DataFrame()
    right_hand = pd.DataFrame()
    
    if results.face_landmarks is not None:
        for i, point in enumerate(results.face_landmarks.landmark):
            face.loc[i, ['x','y','z']] = [point.x, point.y, point.z]

    if results.pose_landmarks is not None:
        for i, point in enumerate(results.pose_landmarks.landmark):
            pose.loc[i, ['x','y','z']] = [point.x, point.y, point.z]


    if results.left_hand_landmarks is not None:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            left_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]


    if results.right_hand_landmarks is not None:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            right_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]

    face = face.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='face')

    pose = pose.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='pose')

    left_hand = left_hand.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='left_hand')

    right_hand = right_hand.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='right_hand')

    landmark = pd.concat([face,pose,left_hand,right_hand]).reset_index(drop=True)
    #merge with lanndmark
    landmark = xyz_skel.merge(landmark, on=['type','landmark_index'], how='left')
    #assign frames to make it unique
    landmark = landmark.assign(frame = frame)
    return landmark

async def do_capture_web_async():
    all_landmarks = []

    cap = cv2.VideoCapture(0)
    xyz = pd.read_parquet('./2044/635217.parquet')
    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
        frame = 0  
        while cap.isOpened():
                #take frame and increment it
            frame+=1
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                    # If loading a video, use 'break' instead of 'continue'.
                continue

                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            #create landmark dataframe
            landmark = create_landmark_frame_df(results, frame, xyz)
            all_landmarks.append(landmark)
            

            # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())

            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_pose_landmarks_style())

            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())

            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())

                # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break
        

    cap.release()
    xyz_np = np.array(all_landmarks).astype(np.float32)

        # Perform prediction
    await predict_and_print(xyz_np)

    

    
# Function to perform prediction and print the result
def predict_and_print(landmark):
    # Load the model and get the prediction_fn
    interpreter = tf.lite.Interpreter(model_path="./model.tflite")
    prediction_fn = interpreter.get_signature_runner("serving_default")

    # Perform prediction
    predictions = prediction_fn(inputs=landmark)

    # Get the predicted sign
    sign = predictions['outputs'].argmax()

    # Print the predicted sign (modify this part according to your requirements)
    print(f"Predicted sign: {sign}")

if __name__ == "__main__":
    try:
        # Create a new asyncio event loop and run the capture task
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(do_capture_web_async())
    except KeyboardInterrupt:
        # If Esc is pressed, stop the capture task and exit gracefully
        print("Process interrupted by Esc key.")
    finally:
        # Release any resources (e.g., close the model, etc.) before exiting
        pass

RuntimeError: Cannot run the event loop while another loop is running

In [2]:
import pandas as pd
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic



def create_landmark_frame_df(results,frame,xyz):
    """
    xyz takes the results from mediapipe and creates a dataframe of the landmark
    
    inputs:
        results: mediapipe results object
        frame: frame number
        xyz: dataframe wof the xyz example data
    
    
    """
    
    #we want the values and rows for every type landmark index so we need skeleton
    xyz_skel = xyz[['type','landmark_index']].drop_duplicates() \
    .reset_index(drop=True).copy()
    
    pose = pd.DataFrame()
    face = pd.DataFrame()
    left_hand = pd.DataFrame()
    right_hand = pd.DataFrame()
    
    if results.face_landmarks is not None:
        for i, point in enumerate(results.face_landmarks.landmark):
            face.loc[i, ['x','y','z']] = [point.x, point.y, point.z]

    if results.pose_landmarks is not None:
        for i, point in enumerate(results.pose_landmarks.landmark):
            pose.loc[i, ['x','y','z']] = [point.x, point.y, point.z]


    if results.left_hand_landmarks is not None:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            left_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]


    if results.right_hand_landmarks is not None:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            right_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]

    face = face.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='face')

    pose = pose.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='pose')

    left_hand = left_hand.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='left_hand')

    right_hand = right_hand.reset_index() \
    .rename(columns={'index': 'landmark_index'}) \
    .assign(type='right_hand')

    landmark = pd.concat([face,pose,left_hand,right_hand]).reset_index(drop=True)
    #merge with lanndmark
    landmark = xyz_skel.merge(landmark, on=['type','landmark_index'], how='left')
    #assign frames to make it unique
    landmark = landmark.assign(frame = frame)
    return landmark


In [3]:
import pandas as pd
import numpy as np
ROWS_PER_FRAME = 543

def load_relevent_data_subset(pq_path):
    data_columns = ['x','y','z']
    data = pd.read_parquet(pq_path,columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME,len(data_columns))
    return data.astype(np.float32)

In [4]:
import cv2
import mediapipe as mp
import tensorflow as tf
import asyncio
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

all_landmarks = []

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    #initially frame is set to 0
    frame = 0 
    while cap.isOpened():
        #increment the frame by 1
        frame+=1
        success, image = cap.read()
        if not success:
              print("Ignoring empty camera frame.")
              # If loading a video, use 'break' instead of 'continue'.
              continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        #passing the example dataset
        xyz = pd.read_parquet('./2044/635217.parquet')
        
        #create landmark dataframe
        
        landmark = create_landmark_frame_df(results, frame, xyz)
        all_landmarks.append(landmark)

        # Draw landmark annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
    
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles
            .get_default_pose_landmarks_style())

        mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())

        mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())
        
        landmark = pd.concat(all_landmarks).reset_index(drop=True)
        
        #load the model
        interpreter = tf.lite.Interpreter("./model.tflite")
        
        #interpret the model
        interpreter = tf.lite.Interpreter(model_path="./model.tflite")

        found_signatures = list(interpreter.get_signature_list().keys())
        
        prediction_fn = interpreter.get_signature_runner("serving_default")
        #pass to prediction_fn
        predictions = prediction_fn(inputs=landmark)
        
        #get the maximum output value
        sign = predictions['outputs'].argmax()
        
        #encode
        train = pd.read_csv('train.csv')

        # Add ordinally Encoded Sign (assign number to each sign name)
        train['sign_ord'] = train['sign'].astype('category').cat.codes

        # Dictionaries to translate sign <-> ordinal encoded sign
        SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
        ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()
        predict_value = ORD2SIGN[sign]
        print(predict_value)

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break
    cap.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


RuntimeError: tensorflow/lite/kernels/reduce.cc:135 current >= 0 && current < input_num_dims was not true.Node number 5 (SUM) failed to prepare.

In [1]:
import numpy as np

# Sample data (list of dictionaries)
data = [
    {'type': 'face', 'landmark_index': 0, 'x': 0.526375, 'y': 0.678677, 'z': -0.034196, 'frame': 1},
    {'type': 'face', 'landmark_index': 1, 'x': 0.523675, 'y': 0.623428, 'z': -0.057393, 'frame': 1},
    # ... (more data)
]

# Define the number of rows per frame
ROWS_PER_FRAME = 543

# Define the data columns
data_columns = ['x', 'y', 'z']

# Calculate the number of frames
n_frames = len(data) // ROWS_PER_FRAME

# Create an empty array to hold the reshaped data
reshaped_data = np.empty((n_frames, ROWS_PER_FRAME, len(data_columns)), dtype=np.float32)

# Iterate through the original data and fill in the reshaped_data array
for i, entry in enumerate(data):
    frame_index = i // ROWS_PER_FRAME
    row_index = i % ROWS_PER_FRAME
    for j, column in enumerate(data_columns):
        reshaped_data[frame_index, row_index, j] = entry[column]

# The reshaped_data array now contains the data in the desired format
print(reshaped_data)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [3]:
data = [
    {'type': 'face', 'landmark_index': 0, 'x': 0.526375, 'y': 0.678677, 'z': -0.034196, 'frame': 1},
    {'type': 'face', 'landmark_index': 1, 'x': 0.523675, 'y': 0.623428, 'z': -0.057393, 'frame': 1},
    # ... (more data)
]
for i, entry in enumerate(data):
   print(entry)

{'type': 'face', 'landmark_index': 0, 'x': 0.526375, 'y': 0.678677, 'z': -0.034196, 'frame': 1}
{'type': 'face', 'landmark_index': 1, 'x': 0.523675, 'y': 0.623428, 'z': -0.057393, 'frame': 1}


In [15]:
import numpy as np

data = [
    {'type': 'face', 'landmark_index': 0, 'x': 0.526375, 'y': 0.678677, 'z': -0.034196, 'frame': 1},
    {'type': 'face', 'landmark_index': 1, 'x': 0.523675, 'y': 0.623428, 'z': -0.057393, 'frame': 1},
    # ... (more data)
]

data_columns = ['x', 'y', 'z']
ROWS_PER_FRAME = 543

# Calculate the number of frames based on the number of rows per frame and the data length
n_frames = (len(data) + ROWS_PER_FRAME - 1) // ROWS_PER_FRAME

# Initialize the reshaped_data array with zeros
reshaped_data = np.zeros((n_frames, ROWS_PER_FRAME, len(data_columns)), dtype=np.float32)

# Iterate through the data and populate the reshaped_data array
for i, entry in enumerate(data):
    frame_index = i // ROWS_PER_FRAME
    row_index = i % ROWS_PER_FRAME
    for j, column in enumerate(data_columns):
        reshaped_data[frame_index, row_index, j] = entry[column]

# Now reshaped_data contains the data in the desired shape
print(reshaped_data)


[[[ 0.526375  0.678677 -0.034196]
  [ 0.523675  0.623428 -0.057393]
  [ 0.        0.        0.      ]
  ...
  [ 0.        0.        0.      ]
  [ 0.        0.        0.      ]
  [ 0.        0.        0.      ]]]


In [16]:
import numpy as np

# Sample data
data = [
    {'type': 'face', 'landmark_index': 0, 'x': 0.526375, 'y': 0.678677, 'z': -0.034196, 'frame': 1},
    {'type': 'face', 'landmark_index': 1, 'x': 0.523675, 'y': 0.623428, 'z': -0.057393, 'frame': 1},
    # ... (more data)
    {'type': 'right_hand', 'landmark_index': 16, 'x': None, 'y': None, 'z': None, 'frame': 24},
    {'type': 'right_hand', 'landmark_index': 17, 'x': None, 'y': None, 'z': None, 'frame': 24},
    # ... (more data)
]

data_columns = ['x', 'y', 'z']
ROWS_PER_FRAME = 543

# Calculate the number of frames based on the number of rows per frame and the data length
n_frames = (len(data) + ROWS_PER_FRAME - 1) // ROWS_PER_FRAME

# Initialize the reshaped_data array with zeros
reshaped_data = np.empty((n_frames, ROWS_PER_FRAME, len(data_columns)), dtype=np.float32)

# Iterate through the data and populate the reshaped_data array
for i, entry in enumerate(data):
    frame_index = i // ROWS_PER_FRAME
    row_index = i % ROWS_PER_FRAME
    if all(entry[column] is None for column in data_columns):
        reshaped_data[frame_index, row_index] = np.nan
    else:
        for j, column in enumerate(data_columns):
            reshaped_data[frame_index, row_index, j] = entry[column]

# Now reshaped_data contains the data in the desired shape, including NaN values
print(reshaped_data)

[[[ 5.263750e-01  6.786770e-01 -3.419600e-02]
  [ 5.236750e-01  6.234280e-01 -5.739300e-02]
  [          nan           nan           nan]
  ...
  [ 8.263918e-37  4.573278e-41  7.566771e-37]
  [ 4.573278e-41  7.565394e-37  4.573278e-41]
  [ 7.160148e-37  4.573278e-41  9.199434e-37]]]


In [1]:
import numpy as np

data = [
    {'type': 'face', 'landmark_index': 0, 'x': 0.526375, 'y': 0.678677, 'z': -0.034196, 'frame': 1},
    {'type': 'face', 'landmark_index': 1, 'x': 0.523675, 'y': 0.623428, 'z': -0.057393, 'frame': 1},
    # ... (more data)
    {'type': 'right_hand', 'landmark_index': 16, 'x': None, 'y': None, 'z': None, 'frame': 24},
    {'type': 'right_hand', 'landmark_index': 17, 'x': None, 'y': None, 'z': None, 'frame': 24},
    # ... (more data)
]

data_columns = ['x', 'y', 'z']
ROWS_PER_FRAME = 543

# Calculate the number of frames based on the number of rows per frame and the data length
n_frames = (len(data) + ROWS_PER_FRAME - 1) // ROWS_PER_FRAME

# Initialize the reshaped_data array with NaN values
reshaped_data = np.empty((n_frames, ROWS_PER_FRAME, len(data_columns)), dtype=np.float32)
reshaped_data[:] = np.nan

# Iterate through the data and populate the reshaped_data array
for i, entry in enumerate(data):
    frame_index = i // ROWS_PER_FRAME
    row_index = i % ROWS_PER_FRAME
    for j, column in enumerate(data_columns):
        if entry[column] is not None:
            reshaped_data[frame_index, row_index, j] = entry[column]

# Now reshaped_data contains the data in the desired shape, including NaN values
print(reshaped_data)


[[[ 0.526375  0.678677 -0.034196]
  [ 0.523675  0.623428 -0.057393]
  [      nan       nan       nan]
  ...
  [      nan       nan       nan]
  [      nan       nan       nan]
  [      nan       nan       nan]]]


In [3]:
import numpy as np
import pandas as pd

# Sample data (based on your provided data)
data = [
    {'type': 'face', 'landmark_index': 0, 'x': 0.553937, 'y': 0.533765, 'z': -0.012798, 'frame': 1},
    {'type': 'face', 'landmark_index': 1, 'x': 0.549591, 'y': 0.488464, 'z': -0.061401, 'frame': 1},
    # ... (more data)
    {'type': 'right_hand', 'landmark_index': 16, 'x': None, 'y': None, 'z': None, 'frame': 24},
    {'type': 'right_hand', 'landmark_index': 17, 'x': None, 'y': None, 'z': None, 'frame': 24},
    # ... (more data)
]

data_columns = ['x', 'y', 'z']
ROWS_PER_FRAME = 543

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Calculate the number of frames based on the number of rows per frame
n_frames = (len(df) + ROWS_PER_FRAME - 1) // ROWS_PER_FRAME

# Initialize the reshaped_data array with NaN values
reshaped_data = np.empty((n_frames, ROWS_PER_FRAME, len(data_columns)), dtype=np.float32)
reshaped_data[:] = np.nan

# Iterate through the data and populate the reshaped_data array
for i, entry in df.iterrows():
    frame_index = i // ROWS_PER_FRAME
    row_index = i % ROWS_PER_FRAME
    for j, column in enumerate(data_columns):
        if pd.notna(entry[column]):
            reshaped_data[frame_index, row_index, j] = entry[column]

# Now reshaped_data contains the data in the desired shape, including NaN values
print(reshaped_data)


[[[ 0.553937  0.533765 -0.012798]
  [ 0.549591  0.488464 -0.061401]
  [      nan       nan       nan]
  ...
  [      nan       nan       nan]
  [      nan       nan       nan]
  [      nan       nan       nan]]]
